# 从神经元到神经网络

[基本神经网络的可视化](http://playground.tensorflow.org)

神经网络是多个“神经元”（感知机）的带权级联，神经网络算法可以提供非线性的复杂模型，它有两个参数：权值矩阵$W^l$和偏置向量$b^l$，不同于感知机的单一向量形式，$W^l$是复数个矩阵，$b^l$是复数个向量，其中的元素分别属于单个层，而每个层的组成单元，就是神经元。

## 神经元

神经网络是由多个“神经元”（感知机）组成的，每个神经元图示如下：

![](cnn0.jpg)

这其实就是一个单层感知机，其输入是由$x_1,x_2,x_3$和+1组成的向量，其输出为$h_{W,b}(x)=f(W^T x)=f(\sum_{i=1}^{3}W_i x_i+b)$，其中$f$是一个激活函数，模拟的是生物神经元在接受一定的刺激之后产生兴奋信号，否则刺激不够的话，神经元保持抑制状态这种现象。这种由一个阈值决定两个极端的函数有点像示性函数，然而这里采用的是Sigmoid函数，其优点是连续可导。

### Sigmoid函数

常用的Sigmoid有两种——

#### 单极性Sigmoid函数

$$f(x)=\frac{1}{1+e^{-x}}$$

其图像如下

![](cnn1.jpg)


#### 双极性Sigmoid函数

$$f(x)=\frac{e^{x}-e^{-x}}{e^{x}+e^{-x}}$$

或者写成

$$f(x)=\frac{1-e^{-x}}{1+e^{-x}}$$

把第一个式子分子分母同时除以$e^x$，令$x=-2x$就得到第二个式子了，换汤不换药。

其图像如下

![](cnn2.jpg)


从它们两个的值域来看，两者名称里的极性应该指的是正负号。从导数来看，它们的导数都非常便于计算：

对于$$f(x)=\frac{1}{1+e^{-x}}$$有$f'(x)=f(x)(1-f(x))$，对于tanh，有$f'(x)=1-(f(x))^2$。

> 1/(1+e^-x)求导的过程：
  $$
\begin{align}
\frac{d}{dx}\sigma&=\frac{d}{dx}(\frac{1}{1+e^{-x}})\\
&=\frac{e^{-x}}{(1+e^{-x})^2}\\
&=\frac{(1+e^{-x})-1}{(1+e^{-x})^2}\\
&=\frac{(1+e^{-x})}{(1+e^{-x})^2}-\frac{1}{(1+e^{-x})^2}\\
&=\sigma(x)-\sigma(x)^2\\
\sigma' &= \sigma(1-\sigma)
\end{align}
  $$
  一旦知道了$f(x)$，就可以直接求$f'(x)$，所以说很方便。

![](cnn7.png)
$$输入层+输出层$$
$$[5,1]$$
本Python实现使用的就是$\frac{1}{1+e^{-x}}$

In [ ]:
def sigmoid(x):
    """
    sigmoid 函数，1/(1+e^-x)
    :param x:
    :return:
    """
    return 1.0/(1.0+math.exp(-x))
 
def dsigmoid(y):
    """
    sigmoid 函数的导数
    :param y:
    :return:
    """
    return y * (1 - y)

也可以使用双曲正切函数tanh

In [ ]:
def sigmoid(x):
    """
    sigmoid 函数，tanh 
    :param x:
    :return:
    """
    return math.tanh(x)

def dsigmoid(y):
    """
    sigmoid 函数的导数
    :param y:
    :return:
    """
    return 1.0 - y ** 2

## 神经网络模型(BPNN)

神经网络就是多个神经元的级联，上一级神经元的输出是下一级神经元的输入，而且信号在两级的两个神经元之间传播的时候需要乘上这两个神经元对应的权值。例如，下图就是一个简单的神经网络：

![](cnn3.jpg)

其中，一共有一个输入层，一个隐藏层和一个输出层。输入层有3个输入节点，标注为+1的那个节点是偏置节点，偏置节点不接受输入，输出总是+1。

这里的数学推导只需关注输入层到隐藏层

定义上标为层的标号，下标为节点的标号，则本神经网络模型的参数是：$(W,b)=(W^{(1)},b^{(1)},W^{(2)},b^{(2)})$，其中$W_{ij}^{(l)}$是第$l$层的第$j$个节点与第$l+1$层第$i$个节点之间的连接参数（或称权值）；$b^{(l)}$表示第$l$层第$i$个偏置节点。这些符号在接下来的前向传播将要用到。

### 前向传播

如果后向传播对应训练的话，那么前向传播就对应预测（分类），并且训练的时候计算误差也要用到预测的输出值来计算误差。

定义$a_i^{(l)}$为第$l$层第$i$个节点的激活值（输出值）。当$l=1$时，$a_i^{(1)}=x_i$。前向传播的目的就是在给定模型参数$W,b$的情况下，计算$l=2,3,4...$层的输出值，直到最后一层就得到最终的输出值。具体怎么算呢，以上图的神经网络模型为例：

$$
\begin{align}
a_1^{(2)}&=f(W_{11}^{(1)}x_1+W_{12}^{(1)}x_2+W_{13}^{(1)}x_3+b_1^{(1)})\\
a_2^{(2)}&=f(W_{21}^{(1)}x_1+W_{22}^{(1)}x_2+W_{23}^{(1)}x_3+b_2^{(1)})\\
a_3^{(2)}&=f(W_{31}^{(1)}x_1+W_{32}^{(1)}x_2+W_{33}^{(1)}x_3+b_3^{(1)})\\
h_{W,b}(x) &= f(a_{1}^{(3)}=f(W_{11}^{(2)}a_{1}^{(2)}+W_{12}^{(2)}a_2^{(2)}+W_{13}^{(2)}a_{3}^{(2)}+b_1^{(2)})
\end{align}
$$

这没什么稀奇的，核心思想是这一层的输出乘上相应的权值加上偏置量代入激活函数等于下一层的输入，一句大白话，所谓中文伪码。

另外，追求好看的话可以把括号里面那个老长老长的加权和定义为一个参数：$z_{i}^{(l)}$表示第$l$层第$i$个节点的输入加权和，比如$z_{i}^{(2)}=\sum_{j=1}^{n}W_{ij}^{(1)}x_j+b_{i}^{(1)}$。那么该节点的输出可以写作$a_{i}^{(l)}=f(z_{i}^{(l)})$。

于是就得到一个好看的形式：

$$
\begin{align}
z^{(2)}&=W^{(1)}x+b^{(1)}\\
a^{(2)}&=f(z^{(2)})\\
z^{(3)}&=W^{(2)}a^{(2)}+b^{(2)}\\
h_{W,b}(x)&=a^{(3)}=f(z^{(3)})
\end{align}
$$

在这个好看的形式下，前向传播可以简明扼要地表示为：

$$
\begin{align}
z^{(l+1)}&=W^{(l)}a^{(l)}+b^{(l)}\\
a^{(l+1)}&=f(z^{(l+1)})
\end{align}
$$

在Python实现中，对应如下方法：

In [ ]:
    def runNN(self, inputs):
        """
        前向传播进行分类
        :param inputs:输入
        :return:类别
        """
        if len(inputs) != self.ni - 1:
            print 'incorrect number of inputs'
 
        for i in range(self.ni - 1):
            self.ai[i] = inputs[i]
 
        for j in range(self.nh):
            sum = 0.0
            for i in range(self.ni):
                sum += ( self.ai[i] * self.wi[i][j] )
            self.ah[j] = sigmoid(sum)
 
        for k in range(self.no):
            sum = 0.0
            for j in range(self.nh):
                sum += ( self.ah[j] * self.wo[j][k] )
            self.ao[k] = sigmoid(sum)
 
        return self.ao

其中，ai、ah、ao分别是输入层、隐藏层、输出层，而wi、wo则分别是输入层到隐藏层、隐藏层到输出层的权值矩阵。在本Python实现中，将偏置量一并放入了矩阵，这样进行线性代数运算就会方便一些。

### 后向传播

后向传播指的是在训练的时候，根据最终输出的误差来调整倒数第二层、倒数第三层……第一层的参数的过程。

#### 符号定义

$x_j^l$：第$l$层第$j$个节点的输入。

$W_{ij}^l$：从第$l-1$层第$i$个节点到第$l$层第$j$个节点的权值。

$\sigma(x)=\frac{1}{1+e^{-x}}$：Sigmoid函数。

$\theta_j^l$：第$l$层第$j$个节点的偏置。

$O_j^l$：第$l$层第$j$个节点的输出。

$t_j$：输出层第$j$个节点的目标值（Target value）。

#### 输出层权值调整

给定训练集$t_k$和模型输出$O_k$（这里没有上标l是因为这里在讨论输出层，l是固定的），输出层的输出误差（或称损失函数吧）定义为：

$$E=\frac{1}{2}\sum_{k\in K}(O_k-t_k)^2$$

其实就是所有实例对应的误差的平方和的一半，训练的目标就是最小化该误差。怎么最小化呢？看损失函数对参数的导数$\frac{\partial E}{\partial W_{jk}^{l}}$呗。

将$E$的定义代入该导数：

$$\frac{\partial E}{\partial W_{jk}}=\frac{\partial}{\partial W_{jk}}\frac{1}{2}\sum_{k\in K}(O_k-t_k)^2$$

无关变量拿出来：

$$\frac{\partial E}{\partial W_{jk}}=(O_k-t_k)\frac{\partial}{\partial W_{jk}}O_k$$

看到这里大概明白为什么非要把误差定义为误差平方和的一半了吧，就是为了好看，数学家都是外貌协会的。

将$O_k=\sigma(x_k)$（输出层的输出等于输入代入Sigmoid函数）这个关系代入有：

$$\frac{\partial E}{\partial W_{jk}}=(O_k-t_k)\frac{\partial}{\partial W_{jk}}\sigma(x_k)$$

对Sigmoid求导有：

$$\frac{\partial E}{\partial W_{jk}}=(O_k-t_k)\sigma(x_k)(1-\sigma(x_k))\frac{\partial}{\partial W_{jk}}x_k$$

要开始耍小把戏了，由于输出层第$k$个节点的输入$x_k$等于上一层第$j$个节点的输出$O_j$乘上$W_{jk}$，即$x_k=O_kW_{jk}$，而上一层的输出$O_j$是与到输出层的权值变量$W_{jk}$无关的，可以看做一个常量，是线性关系。所以对$x_k$求权值变量$W_{jk}$的偏导数直接等于$O_j$，也就是说：$\frac{\partial}{\partial W_{jk}}x_k=\frac{\partial}{\partial W_{jk}}(O_j W_{jk})=O_j$。

然后将上面用过的$\sigma(x_k)=O_k$代进去就得到最终的：

$$\frac{\partial E}{\partial W_{jk}}=(O_k-t_k)O_k(1-O_k)O_j$$

为了表述方便将上式记作：

$$\frac{\partial E}{\partial W_{jk}}=O_k\delta_k$$

其中：

$$\delta=(O_k-t_k)O_k(1-O_k)O_j$$



#### 隐藏层权值调整

依然采用类似的方法求导，只不过求的是关于隐藏层和前一层的权值参数的偏导数：

$$\frac{\partial E}{\partial W_{ij}}=\frac{\partial}{\partial W_{ij}}\frac{1}{2}\sum_{k\in K}(O_k-t_k)^2$$


老样子：

$$\frac{\partial E}{\partial W_{ij}}=\sum_{k\in K}(O_k-t_k)\frac{\partial}{\partial W_{ij}}O_k$$


还是老样子：

$$\frac{\partial E}{\partial W_{ij}}=\sum_{k\in K}(O_k-t_k)\frac{\partial}{\partial W_{ij}}\sigma(x_k)$$

还是把Sigmoid弄进去：

$$\frac{\partial E}{\partial W_{ij}}=\sum_{k\in K}(O_k-t_k)\sigma(x_k)(1-\sigma(x_k))\frac{\partial x_k}{\partial W_{ij}}$$

把$\sigma(x_k)=O_k$代进去，并且将导数部分拆开：

$$\frac{\partial E}{\partial W_{ij}}=\sum_{k\in K}(O_k-t_k)O_k(1-O_k)\frac{\partial x_k}{\partial O_{j}}\cdot\frac{\partial O_j}{\partial W_{ij}}$$


又要耍把戏了，输出层的输入等于上一层的输出乘以相应的权值，亦即$x_k=W_{jk}O_j$，于是得到：

$$\frac{\partial E}{\partial W_{ij}}=\sum_{k\in K}(O_k-t_k)O_k(1-O_k)W_{jk}\frac{\partial O_j}{\partial W_{ij}}$$

把最后面的导数挪到前面去，接下来要对它动刀了：

$$\frac{\partial E}{\partial W_{ij}}=\frac{\partial O_j}{\partial W_{ij}}\sum_{k\in K}(O_k-t_k)O_k(1-O_k)W_{jk}$$

再次利用$O_k=\sigma(x_k)$，这对$j$也成立，代进去：

$$\frac{\partial E}{\partial W_{ij}}=O_j(1-O_j)\frac{\partial x_j}{\partial W_{ij}}\sum_{k\in K}(O_k-t_k)O_k(1-O_k)W_{jk}$$

再次利用$x_k=W_{jk}O_j$，$j$换成$i$,$k$换成$j$也成立，代进去：

$$\frac{\partial E}{\partial W_{ij}}=O_j(1-O_j)O_i\sum_{k\in K}(O_k-t_k)O_k(1-O_k)W_{jk}$$

利用刚才定义的$\delta_k$，最终得到：

$$\frac{\partial E}{\partial W_{ij}}=O_iO_j(1-O_j)\sum_{k\in K}\delta_k W_{jk}$$

其中：

$$\delta=O_k(1-O_k)(O_k-t_k)$$

我们还可以仿照$\delta_k$的定义来定义一个$\delta_j$，得到：

$$\frac{\partial E}{\partial W_{ij}}=O_i\delta_j$$

其中

$$\delta_j=O_j(1-O_j)\sum_{k\in K}\delta_k W_{jk}$$

#### 偏置的调整

因为没有任何节点的输出流向偏置节点，所以偏置节点不存在上层节点到它所对应的权值参数，也就是说不存在关于权值变量的偏导数。虽然没有流入，但是偏置节点依然有输出（总是+1），该输出到下一层某个节点的时候还是会有权值的，对这个权值依然需要更新。

我们可以直接对偏置求导，发现：

$$\frac{\partial O}{\partial \theta}=O(1-O)\frac{\partial \theta}{\partial \theta}$$

原视频中说$\frac{\partial O}{\partial \theta}=1$，这是不对的，作者也在讲义中修正了这个错误，$\frac{\partial O}{\partial \theta}=O(1–O)$。

然后再求$\frac{\partial E}{\partial \theta}$，$\frac{\partial E}{\partial \theta}=\sum_{k \in K}(O_k-t_k)\frac{\partial}{\partial \theta}O_k$，后面的导数等于$\frac{\partial O}{\partial \theta}=O(1-O)$，代进去有

$$\frac{\partial E}{\partial \theta}=\delta_l$$

其中，

$$\delta_k=O_k(1-O_k)(O_k-t_k)$$

#### 后向传播算法步骤

- 随机初始化参数，对输入利用前向传播计算输出。

- 对每个输出节点按照下式计算$\delta$：$\delta_k=O_k(1-O_k)(O_k-t_k)$

- 对每个隐藏节点按照下式计算$\delta$：$\delta_j=O_j(1-O_j)\sum_{k\in K}\delta_k W_{jk}$

- 计算梯度$\Delta W=-\eta\delta_lO_{l-1},\Delta \theta=-\eta\delta_l$，并更新权值参数和偏置参数：$W+\Delta W\to W,\theta+\Delta \theta \to \theta$。这里的$\eta$是学习率，影响训练速度。

#### 后向传播算法实现

In [ ]:
    def backPropagate(self, targets, N, M):
        """
        后向传播算法
        :param targets: 实例的类别 
        :param N: 本次学习率
        :param M: 上次学习率
        :return: 最终的误差平方和的一半
        """
        # http://www.youtube.com/watch?v=aVId8KMsdUU&feature=BFa&list=LLldMCkmXl4j9_v0HeKdNcRA
 
        # 计算输出层 deltas
        # dE/dw[j][k] = (t[k] - ao[k]) * s'( SUM( w[j][k]*ah[j] ) ) * ah[j]
        output_deltas = [0.0] * self.no
        for k in range(self.no):
            error = targets[k] - self.ao[k]
            output_deltas[k] = error * dsigmoid(self.ao[k])
 
        # 更新输出层权值
        for j in range(self.nh):
            for k in range(self.no):
                # output_deltas[k] * self.ah[j] 才是 dError/dweight[j][k]
                change = output_deltas[k] * self.ah[j]
                self.wo[j][k] += N * change + M * self.co[j][k]
                self.co[j][k] = change
 
        # 计算隐藏层 deltas
        hidden_deltas = [0.0] * self.nh
        for j in range(self.nh):
            error = 0.0
            for k in range(self.no):
                error += output_deltas[k] * self.wo[j][k]
            hidden_deltas[j] = error * dsigmoid(self.ah[j])
 
        # 更新输入层权值
        for i in range(self.ni):
            for j in range(self.nh):
                change = hidden_deltas[j] * self.ai[i]
                # print 'activation',self.ai[i],'synapse',i,j,'change',change
                self.wi[i][j] += N * change + M * self.ci[i][j]
                self.ci[i][j] = change
 
        # 计算误差平方和
        # 1/2 是为了好看，**2 是平方
        error = 0.0
        for k in range(len(targets)):
            error = 0.5 * (targets[k] - self.ao[k]) ** 2
        return error

注意不同于上文的单一学习率$\eta$，这里有两个学习率$N$和$M$。$N$相当于上文的$\eta$，而$M$则是在用上次训练的梯度更新权值时的学习率。这种同时考虑最近两次迭代得到的梯度的方法，可以看做是对单一学习率的改进。

> 这里并没有出现任何更新偏置的操作，为什么？

> 因为这里的偏置是单独作为一个偏置节点放到输入层里的，它的值（输出，没有输入）固定为1，它的权值已经自动包含在上述权值调整中了。
  如果将偏置作为分别绑定到所有神经元的许多值，那么则需要进行偏置调整，而不需要权值调整（此时没有偏置节点）。
  哪个方便，当然是前者了，这也导致了大部分神经网络实现都采用前一种做法。

### 完整的实现

![](cnn6.png)

$$输入层+隐藏层+输出层$$
$$[4,4,1]$$
#### 对象式实现

[完整的实现]()

直接运行bpnn.py即可得到输出：

```shell
Combined error 0.171204877501
Combined error 0.190866985872
Combined error 0.126126875154
Combined error 0.0658488960415
Combined error 0.0353249077599
Combined error 0.0214428399072
Combined error 0.0144886807614
Combined error 0.0105787745309
Combined error 0.00816264126944
Combined error 0.00655731212209
Combined error 0.00542964723539
Combined error 0.00460235328667
Combined error 0.00397407912435
Combined error 0.00348339081276
Combined error 0.00309120476889
Combined error 0.00277163178862
Combined error 0.00250692771135
Combined error 0.00228457151714
Combined error 0.00209550313514
Combined error 0.00193302192499
Inputs: [0, 0] --> [0.9982333356008245] 	Target [1]
Inputs: [0, 1] --> [0.9647325217906978] 	Target [1]
Inputs: [1, 0] --> [0.9627966274767186] 	Target [1]
Inputs: [1, 1] --> [0.05966109502803293] 	Target [0]
```
IBM利用Neil Schemenauer的这一模块（旧版）做了一个识别代码语言的例子，我将其更新到新版，已经整合到了项目中。

要运行测试的话，执行命令

```shell
code_recognizer.py testdata.200
```

即可得到输出：

```shell
ERROR_CUTOFF = 0.01
INPUTS = 20
ITERATIONS = 1000
MOMENTUM = 0.1
TESTSIZE = 500
OUTPUTS = 3
TRAINSIZE = 500
LEARNRATE = 0.5
HIDDEN = 8
Targets: [1, 0, 0] -- Errors: (0.000 OK)   (0.001 OK)   (0.000 OK)   -- SUCCESS!
```

值得一提的是，这里的HIDDEN = 8指的是隐藏层的节点个数，不是层数，层数多了就变成DeepLearning了。

#### 函数式实现

接下来秀下操作，比如，只用9行

尽管我们不直接用神经网络库，但还是要从Python数学库Numpy中导入4种方法：

- exp： 自然对常数
- array： 创建矩阵
- dot：矩阵乘法
- random： 随机数

In [5]:
from numpy import exp, array, random, dot
training_set_inputs = array([[0, 0, 1], [1, 1, 1], [1, 0, 1], [0, 1, 1]])
training_set_outputs = array([[0, 1, 1, 0]]).T
random.seed(1)
synaptic_weights = 2 * random.random((3, 1)) - 1
for iteration in range(10000):
    output = 1 / (1 + exp(-(dot(training_set_inputs, synaptic_weights))))
    synaptic_weights += dot(training_set_inputs.T, (training_set_outputs - output) * output * (1 - output))
print("accurency:",1 / (1 + exp(-(dot(array([1, 0, 0]), synaptic_weights)))))

accurency: [ 0.99993704]


In [7]:
from numpy import exp, array, random, dot

class NeuralNetwork():
    def __init__(self):
        # 随机数发生器种子，以保证每次获得相同结果
        random.seed(1)

        # 对单个神经元建模，含有3个输入连接和一个输出连接
        # 对一个3 x 1的矩阵赋予随机权重值。范围-1～1，平均值为0
        self.synaptic_weights = 2 * random.random((3, 1)) - 1

    # Sigmoid函数，S形曲线
    # 用这个函数对输入的加权总和做正规化，使其范围在0～1
    def __sigmoid(self, x):
        return 1 / (1 + exp(-x))

    # Sigmoid函数的导数
    # Sigmoid曲线的梯度
    # 表示我们对当前权重的置信程度
    def __sigmoid_derivative(self, x):
        return x * (1 - x)

    # 通过试错过程训练神经网络
    # 每次都调整突触权重
    def train(self, training_set_inputs, training_set_outputs, number_of_training_iterations):
        for iteration in range(number_of_training_iterations):
            # 将训练集导入神经网络
            output = self.think(training_set_inputs)

            # 计算误差（实际值与期望值之差）
            error = training_set_outputs - output

            # 将误差、输入和S曲线梯度相乘
            # 对于置信程度低的权重，调整程度也大
            # 为0的输入值不会影响权重
            adjustment = dot(training_set_inputs.T, error * self.__sigmoid_derivative(output))

            # 调整权重
            self.synaptic_weights += adjustment

    # 神经网络一思考
    def think(self, inputs):
        # 把输入传递给神经网络
        return self.__sigmoid(dot(inputs, self.synaptic_weights))


if __name__ == "__main__":

    # 初始化神经网络
    neural_network = NeuralNetwork()

    print("随机的初始突触权重：")
    print(neural_network.synaptic_weights)

    # 训练集。四个样本，每个有3个输入和1个输出
    training_set_inputs = array([[0, 0, 1], [1, 1, 1], [1, 0, 1], [0, 1, 1]])
    training_set_outputs = array([[0, 1, 1, 0]]).T

    # 用训练集训练神经网络
    # 重复一万次，每次做微小的调整
    neural_network.train(training_set_inputs, training_set_outputs, 10000)

    print("训练后的突触权重：")
    print(neural_network.synaptic_weights)

    # 用新数据测试神经网络
    print("考虑新的形势 [1, 0, 0] -> ?: ")
    print(neural_network.think(array([1, 0, 0])))

随机的初始突触权重：
[[-0.16595599]
 [ 0.44064899]
 [-0.99977125]]
训练后的突触权重：
[[ 9.67299303]
 [-0.2078435 ]
 [-4.62963669]]
考虑新的形势 [1, 0, 0] -> ?: 
[ 0.99993704]


#### 多层隐藏层实现
![](cnn5.png)
$$输入层+3*隐藏层+输出层$$
$$[4,4,4,4,1]$$

In [9]:
from numpy import exp, array, random, dot


class NeuronLayer():
    def __init__(self, number_of_neurons, number_of_inputs_per_neuron):
        self.synaptic_weights = 2 * random.random((number_of_inputs_per_neuron, number_of_neurons)) - 1


class NeuralNetwork():
    def __init__(self, layer1, layer2):
        self.layer1 = layer1
        self.layer2 = layer2

    # Sigmoid函数，S形曲线
    # 传递输入的加权和，正规化为0-1
    def __sigmoid(self, x):
        return 1 / (1 + exp(-x))

    # Sigmoid函数的导数，Sigmoid曲线的梯度，表示对现有权重的置信程度
    def __sigmoid_derivative(self, x):
        return x * (1 - x)

    # 通过试错训练神经网络，每次微调突触权重
    def train(self, training_set_inputs, training_set_outputs, number_of_training_iterations):
        for iteration in range(number_of_training_iterations):
            # 将整个训练集传递给神经网络
            output_from_layer_1, output_from_layer_2 = self.think(training_set_inputs)

            # 计算第二层的误差
            layer2_error = training_set_outputs - output_from_layer_2
            layer2_delta = layer2_error * self.__sigmoid_derivative(output_from_layer_2)

            # 计算第一层的误差，得到第一层对第二层的影响
            layer1_error = layer2_delta.dot(self.layer2.synaptic_weights.T)
            layer1_delta = layer1_error * self.__sigmoid_derivative(output_from_layer_1)

            # 计算权重调整量
            layer1_adjustment = training_set_inputs.T.dot(layer1_delta)
            layer2_adjustment = output_from_layer_1.T.dot(layer2_delta)

            # 调整权重
            self.layer1.synaptic_weights += layer1_adjustment
            self.layer2.synaptic_weights += layer2_adjustment

    # 神经网络一思考
    def think(self, inputs):
        output_from_layer1 = self.__sigmoid(dot(inputs, self.layer1.synaptic_weights))
        output_from_layer2 = self.__sigmoid(dot(output_from_layer1, self.layer2.synaptic_weights))
        return output_from_layer1, output_from_layer2

    # 输出权重
    def print_weights(self):
        print("    Layer 1 (4 neurons, each with 3 inputs): ")
        print(self.layer1.synaptic_weights)
        print("    Layer 2 (1 neuron, with 4 inputs):")
        print(self.layer2.synaptic_weights)

        
if __name__ == "__main__":

    # 设定随机数种子
    random.seed(1)

    # 创建第一层 (4神经元, 每个3输入)
    layer1 = NeuronLayer(4, 3)

    # 创建第二层 (单神经元，4输入)
    layer2 = NeuronLayer(1, 4)

    # 组合成神经网络
    neural_network = NeuralNetwork(layer1, layer2)

    print("Stage 1) 随机初始突触权重： ")
    neural_network.print_weights()

    # 训练集，7个样本，均有3输入1输出
    training_set_inputs = array([[0, 0, 1], [0, 1, 1], [1, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 1], [0, 0, 0]])
    training_set_outputs = array([[0, 1, 1, 1, 1, 0, 0]]).T

    # 用训练集训练神经网络
    # 迭代60000次，每次微调权重值
    neural_network.train(training_set_inputs, training_set_outputs, 60000)

    print("Stage 2) 训练后的新权重值： ")
    neural_network.print_weights()

    # 用新数据测试神经网络
    print("Stage 3) 思考新形势 [1, 1, 0] -> ?: ")
    hidden_state, output = neural_network.think(array([1, 1, 0]))
    print(output)

Stage 1) 随机初始突触权重： 
    Layer 1 (4 neurons, each with 3 inputs): 
[[-0.16595599  0.44064899 -0.99977125 -0.39533485]
 [-0.70648822 -0.81532281 -0.62747958 -0.30887855]
 [-0.20646505  0.07763347 -0.16161097  0.370439  ]]
    Layer 2 (1 neuron, with 4 inputs):
[[-0.5910955 ]
 [ 0.75623487]
 [-0.94522481]
 [ 0.34093502]]
Stage 2) 训练后的新权重值： 
    Layer 1 (4 neurons, each with 3 inputs): 
[[ 0.3122465   4.57704063 -6.15329916 -8.75834924]
 [ 0.19676933 -8.74975548 -6.1638187   4.40720501]
 [-0.03327074 -0.58272995  0.08319184 -0.39787635]]
    Layer 2 (1 neuron, with 4 inputs):
[[ -8.18850925]
 [ 10.13210706]
 [-21.33532796]
 [  9.90935111]]
Stage 3) 思考新形势 [1, 1, 0] -> ?: 
[ 0.0078876]


## 比较不同层的神经网络

![](cnn4.jpg)

数学家们经过严格的数学证明，双隐层神经网络能够解决任意复杂的分类问题。

## 神经网络的统一解法(分类问题)

对于一切分类问题我们都可以有一个统一的方法，只需要两层隐藏层。以下面这个3分类问题为例

![](cnn7.jpg)

我们通过取”半平面”、取交集生成精确包含每一个样本点的凸域（所以凸域的个数与训练集的样本的个数相等），再对同类的样本点的区域取并集，这样无论多复杂的分离界面我们可以考虑进去。

于是，我们设计双隐层神经网络结构如下，每一层的节点数依次为：3、25、7、3:

![](cnn6.jpg)

解释下每一层的作用： 

1. 第一层是`输入层`，对应着每一个输入的二维样本点x(X1,X2)的2个输坐标和1个辅助偏移的常量“1”，总共3个节点。 

2. 第二层是`第一层的输出层`，每个节点对应着输入样本点属于某个”半平面”的概率，该”半平面”的正向法向量就对应着输入层（输入样本点坐标）到该节点的权重w和偏移b。
  为了精确定位总共这6个点，我们围绕每个点附近平行于坐标轴切4刀，用四个”半平面”相交产生的方形封闭凸域精确包裹定位该点，所以这一层对应着总共产生了的4*6=24个”半平面”，有24个节点。
  第二层同时也是第三层的输入层，要考虑再加上求第三层节点时要用的1个辅助偏移的常量节点“1”，第二层总共有24+1=25个节点。 

3. 第三层是`第二层的输出层`，每个节点对应着输入样本点属于第二层区分出来的某4个”半平面”取交集形成的某个方形封闭凸域的概率，故总共需要24/4=6个节点。
  因为采用我们的划分方式，只需要四个”半平面”就可以精确包裹某一个样本点了，我们认为其他的”半平面”贡献的权重应该为0，就只画了生成该凸域的四个”半平面”给予的权重对应的连线，其他连线不画。
  第三层同时也是第四层的输入层，要考虑再加上求第四层节点时要用的1个辅助偏移的常量节点“1”，第三层总共有6+1=7个节点。 

4. 第四层是`第三层的输出层`，每个节点对应着输入样本点属于某一类点所在的所有区域的概率。
  为找到该类的区分区域，对第三层属于同一类点的2个凸域的2个节点取并集，故总共需要6/2=3个节点。
  （同样，我们不画其他类的凸域贡献的权重的连线） 
  
小结一下就是先取”半平面”再取交集最后取并集。 

一图胜千言

![](cnn5.jpg)

# 从BPNN到CNN

## CNN的层级结构

[CNN分类任务可视化](http://shixialiu.com/publications/cnnvis/demo/)

![](cnn8.png)

从上图我们可以看到，CNN的架构和基本的神经网络差不多（事实上，CNN的前面几层是特殊的卷积层，图中表示不明显）

具体一点，卷积神经网络各个层级结构如下：

![](cnn0.jpeg)

上图中CNN要做的事情是：给定一张图片，是车还是马未知，是什么车也未知，现在需要模型判断这张图片里具体是一个什么东西，总之输出一个结果：如果是车 那是什么车

所以

- 最左边是
  - 数据输入层，对数据做一些处理(数据预处理)，
    - 去均值（把输入数据各个维度都中心化为0，避免数据过多偏差，影响训练效果）、
    - 归一化（把所有的数据都归一到同样的范围）、
    - PCA/白化等等。CNN只对训练集做“去均值”这一步。

- 中间是
  - CONV：卷积计算层，线性乘积 求和。
  - RELU：激励层，上文2.2节中有提到：ReLU是激活函数的一种,也有用Sigmoid的。
  - POOL：池化层，简言之，即取区域平均或最大。

- 最右边是
  - FC：全连接层，和基本神经网络一模一样
    
这几个部分中，卷积计算层是CNN的核心，下文将重点阐述。


### 卷积层

从上面的讨论我们知道，对一般的数据分类任务，几层的BPNN就可以搞定（不然再加层加神经元，总会成功的）

但是，如果我们分类的对象是图片呢？

在计算机中图片表达为矩阵，矩阵的数代表灰度

![](cnn0.gif)

图像在计算机中是一堆按顺序排列的数字，数值为0到255。0表示最暗，255表示最亮。

你可以把这堆数字用一个长长的向量来表示，也就是[tensorflow的mnist教程](https://www.tensorflow.org/get_started/mnist/beginners)中784维向量的表示方式。 然而这样会`失去平面结构的信息`，为保留该结构信息，通常选择矩阵的表示方式：28x28的矩阵。

上图是只有黑白颜色的灰度图，而更普遍的图片表达方式是RGB颜色模型，即红（Red）、绿（Green）、蓝（Blue）三原色的色光以不同的比例相加，以产生多种多样的色光。这样，RGB颜色模型中，单个矩阵就扩展成了有序排列的三个矩阵，也可以用三维张量去理解，其中的每一个矩阵又叫这个图片的一个channel。

在电脑中，一张图片是数字构成的“长方体”。可用 宽width, 高height, 深depth 来描述，如图。

![](cnn0.png)

画面识别的输入x是shape为(width, height, depth)的三维张量。

比如下面x.shape=(width=5,height=2,depth=3)

```python
x=[
 [[1,2,3,4,5],
  [6,7,8,9,0]],
 
 [[1,2,3,4,5],
  [6,7,8,9,0]],
  
 [[1,2,3,4,5],
  [6,7,8,9,0]]
  ]
```

接下来要考虑的就是该如何处理这样的“数字长方体”。

#### CNN怎么进行识别

当我们给定一个"X"的图案，计算机怎么识别这个图案就是“X”呢？一个可能的办法就是计算机存储一张标准的“X”图案，然后把需要识别的未知图案跟标准"X"图案进行比对，如果二者一致，则判定未知图案即是一个"X"图案。

而且即便未知图案可能有一些平移或稍稍变形，依然能辨别出它是一个X图案。如此，CNN是把未知图案和标准X图案一个局部一个局部的对比，如下图所示

![](cnn8.jpg)

而未知图案的局部和标准X图案的局部一个一个比对时的计算过程，便是卷积操作。卷积计算结果为1表示匹配，否则不匹配。

举个数学例子

$$x=\begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9
\end{bmatrix},y=\begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9
\end{bmatrix},x与y的卷积为\begin{bmatrix}
1*1 &+ 2*2 &+ 3*3 \\
+4*4 &+ 5*5 &+ 6*6 \\
+7*7 &+ 8*8 &+ 9*9
\end{bmatrix}=\begin{bmatrix}
1 + 4 + 9 + 16 + 25 + 36 + 49 + 64 + 81
\end{bmatrix}=285
$$
两个矩阵的卷积是一个数！这个数非常大，所以匹配。更多时候我们会根据平均值把图象二值化，只留0和1,这样计算和判断更准确

接下来，我们来了解下什么是卷积操作。

#### 数学上的卷积

> 对图像（不同的数据窗口数据）和滤波矩阵（一组固定的权重：因为每个神经元的多个权重固定，所以又可以看做一个恒定的滤波器filter）做内积（逐个元素相乘再求和）的操作就是所谓的『卷积』操作，也是卷积神经网络的名字来源。

非严格意义上来讲，下图中红框框起来的部分便可以理解为一个滤波器，即带着一组固定权重的神经元。多个滤波器叠加便成了卷积层。
![](cnn9.jpg)
换句话说，滤波器相当于平面状态的权重，训练CNN就是训练这些滤波器的权重

OK，举个具体的例子。比如下图中，图中左边部分是原始输入数据，图中中间部分是滤波器filter，图中右边是输出的新的二维数据。

![](cnn9.png)
	
中间滤波器filter与数据窗口做内积，其具体计算过程则是：
$$pic=\begin{bmatrix}
0 & 0 & 0 \\
0 & 1 & 1 \\
0 & 1 & 2
\end{bmatrix},y=\begin{bmatrix}
-4 & 0 & 0 \\
0 & 0 & 0 \\
0 & 0 & -4
\end{bmatrix},x与y的卷积为\begin{bmatrix}
4*0 + 0*0 + 0*0 + 0*0 + 0*1 + 0*1 + 0*0 + 0*1 + -4*2
\end{bmatrix}=-8
$$


#### 图像上的卷积

在下图对应的计算过程中，输入是一定区域大小(width*height)的数据，和滤波器filter（带着一组固定权重的神经元）做内积后等到新的二维数据。

具体来说，左边是图像输入，中间部分就是滤波器filter（带着一组固定权重的神经元），不同的滤波器filter会得到不同的输出数据，比如颜色深浅、轮廓。相当于如果想提取图像的不同特征，则用不同的滤波器filter，提取想要的关于图像的特定信息：颜色深浅或轮廓。

如下图所示

![](cnn10.png)

在CNN中，滤波器filter（带着一组固定权重的神经元）对局部输入数据进行卷积计算。每计算完一个数据窗口内的局部数据后，数据窗口不断平移滑动，直到计算完所有数据。这个过程中，有这么几个参数

1. 深度depth：神经元个数，决定输出的depth厚度。同时代表滤波器个数。
2. 步长stride：决定滑动多少步可以到边缘。
3. 填充值zero-padding：在外围边缘补充若干圈0，方便从初始位置以步长为单位可以刚好滑倒末尾位置，通俗地讲就是为了总长能被步长整除。 

![](cnn11.png)

In [9]:
from IPython.core.display import HTML
HTML('<iframe src="cnn.html" width="100%" height="700px;" style="border:none;"></iframe>')

可以看到：
- 两个神经元，即depth=2，意味着有两个滤波器。
- 数据窗口每次移动两个步长取3*3的局部数据，即stride=2。
- zero-padding=1。

然后分别以两个滤波器filter为轴滑动数组进行卷积计算，得到两组不同的结果。

如果初看上图，可能不一定能立马理解啥意思，但结合上文的内容后，理解这个动图已经不是很困难的事情：

- 左边是输入（7*7*3中，7*7代表图像的像素/长宽，3代表R、G、B 三个颜色通道）
- 中间部分是两个不同的滤波器Filter w0、Filter w1
- 最右边则是两个不同的输出

- 随着左边数据窗口的平移滑动，滤波器Filter w0 / Filter w1对不同的局部数据进行卷积计算。

值得一提的是：

1. 左边数据在变化，每次滤波器都是针对某一局部的数据窗口进行卷积，这就是所谓的CNN中的局部感知机制。
  - 打个比方，滤波器就像一双眼睛，人类视角有限，一眼望去，只能看到这世界的局部。如果一眼就看到全世界，你会累死，而且一下子接受全世界所有信息，你大脑接收不过来。当然，即便是看局部，针对局部里的信息人类双眼也是有偏重、偏好的。比如看美女，对脸、胸、腿是重点关注，所以这3个输入的权重相对较大。

2. 与此同时，数据窗口滑动，导致输入在变化，但中间滤波器Filter w0的权重（即每个神经元连接数据窗口的权重）是固定不变的，这个权重不变即所谓的CNN中的参数（权重）共享机制。
  - 打个比方，某人环游全世界，所看到的信息在变，但采集信息的双眼不变。btw，不同人的双眼 看同一个局部信息 所感受到的不同，即一千个读者有一千个哈姆雷特，所以不同的滤波器 就像不同的双眼，不同的人有着不同的反馈结果。
  
嗯，在这里可以停一下，去看上面的动图，仔细揣摩卷积操作到底怎么进行的

### 激励层

#### ReLU激励层

我们介绍过激活函数sigmoid，但实际梯度下降中，sigmoid容易饱和、造成终止梯度传递，且没有0中心化。咋办呢，可以尝试另外一个激活函数：ReLU，其图形表示如下

![](cnn12.png)

 ReLU的优点是收敛快，求梯度简单。
 
### 池化层

前头说了，池化，简言之，即取区域平均或最大，如下图所示

![](cnn13.png)

上图所展示的是取区域最大，即上图左边部分中 左上角2x2的矩阵中6最大，右上角2x2的矩阵中8最大，左下角2x2的矩阵中3最大，右下角2x2的矩阵中4最大，所以得到上图右边部分的结果：6 8 3 4。很简单不是？

其实池化除取区域最大，区域平均外，也可以用$1*1$的卷积核来取，甚至可以用另一个神经网络来取

## CNN与画面

### 画面不变性

我们知道一个物体不管在画面左侧还是右侧，都会被识别为同一物体，这一特点就是不变性（invariance），如下图所示。

![](cnn1.png)

我们希望所建立的网络可以尽可能的满足这些不变性特点。

为了理解卷积神经网络对这些不变性特点的贡献，我们将用不具备这些不变性特点的前馈神经网络来进行比较。

#### 平移不变性

![](cnn13.png)

方便起见，我们用depth只有1的灰度图来举例。

想要完成的任务是：在宽长为4x4的图片中识别是否有下图所示的“横折”。 图中，黄色圆点表示值为0的像素，深色圆点表示值为1的像素。 我们知道不管这个横折在图片中的什么位置，都会被认为是相同的横折。

![](cnn2.png)

##### 图片识别--前馈神经网络

若训练前馈神经网络来完成该任务，那么表达图像的三维张量将会被摊平成一个向量，作为网络的输入，即(width, height, depth)为(4, 4, 1)的图片会被展成维度为16的向量作为网络的输入层。再经过几层不同节点个数的隐藏层，最终输出两个节点，分别表示“有横折的概率”和“没有横折的概率”.

我们用数字（16进制）对图片中的每一个像素点（pixel）进行编号。 

当使用右侧那种物体位于中间的训练数据来训练网络时，网络就只会对编号为5,6,9,a的节点的权重进行调节。 

若让该网络识别位于右下角的“横折”时，则无法识别。换句话说，前馈神经网络很难满足平移不变性，对二维图象的感知有限。

![](cnn3.png)

##### 图片识别--卷积神经网络

卷积神经网络对二维图象的特征提取有2个策略：局部连接，空间共享

###### 局部连接

就是filter不和整个矩阵全连接而是通过局部卷积和滑动来提取特征，同时满足了平移不变性

###### 空间共享

有２个方面

- 一个偏值节点对一个filter在任意位置的共享
- pooling层，一个filter在不同位置的卷积共享为最大或平均

#### 旋转和视角不变性

![](cnn15.png)

个人觉得卷积神经网络克服这一不变性的主要手段还是靠大量的数据。 并没有明确加入“旋转和视角不变性”的先验特性。

#### 尺寸不变性

与平移不变性不同，最初的卷积网络并没有明确照顾尺寸不变性这一特点。

我们知道filter的size是事先选择的，而不同的尺寸所寻找的形状（概念）范围不同。

从直观上思考，如果选择小范围，再一步步通过组合，仍然是可以得到大范围的形状。 如3x3尺寸的形状都是可以由2x2形状的图形组合而成。所以形状的尺寸不变性对卷积神经网络而言并不算问题。 这恐怕ZF Net让第一层的stride和filter size更小，VGGNet将所有filter size都设置成3x3仍可以得到优秀结果的一个原因。

但是，除了形状之外，很多概念的抓取通常需要考虑一个像素与周边更多像素之间的关系后得出。 也就是说5x5的filter也是有它的优点。 同时，小尺寸的堆叠需要很多个filters来共同完成，如果需要抓取的形状恰巧在5x5的范围，那么5x5会比3x3来的更有效率。 所以一次性使用多个不同filter size来抓取多个范围不同的概念是一种顺理成章的想法，而这个也就是Inception。 可以说Inception是为了尺寸不变性而引入的一个先验知识。

##### Inception

![](cnn16.png)

上图是Inception的结构，尽管也有不同的版本，但是其动机都是一样的：消除尺寸对于识别结果的影响，一次性使用多个不同filter size来抓取多个范围不同的概念，并让网络自己选择需要的特征。

你也一定注意到了蓝色的1x1卷积，撇开它，先看左边的这个结构。

###### 左边结构

输入（可以是被卷积完的长方体输出作为该层的输入）进来后，通常我们可以选择直接使用像素信息(1x1卷积)传递到下一层，可以选择3x3卷积，可以选择5x5卷积，还可以选择max pooling的方式downsample刚被卷积后的feature maps。 但在实际的网络设计中，究竟该如何选择需要大量的实验和经验的。 Inception就不用我们来选择，而是将4个选项给神经网络，让网络自己去选择最合适的解决方案。

接下来我们再看右边的这个结构，多了很多蓝色的1x1卷积。 这些1x1卷积的作用是为了让网络根据需要能够更灵活的控制数据的depth的。

###### 1x1卷积核

如果卷积的输出输入都只是一个平面，那么1x1卷积核并没有什么意义，它是完全不考虑像素与周边其他像素关系。 但卷积的输出输入是长方体，所以1x1卷积实际上是对每个像素点，在不同的channels上进行线性组合（信息整合），且保留了图片的原有平面结构，调控depth，从而完成升维或降维的功能。

如下图所示，如果选择2个filters的1x1卷积层，那么数据就从原本的depth 3 降到了2。若用4个filters，则起到了升维的作用。

这就是为什么上面Inception的4个选择中都混合一个1x1卷积，如右侧所展示的那样。 其中，绿色的1x1卷积本身就1x1卷积，所以不需要再用另一个1x1卷积。 而max pooling用来去掉卷积得到的Feature Map中的冗余信息，所以出现在1x1卷积之前，紧随刚被卷积后的feature maps。（由于没做过实验，不清楚调换顺序会有什么影响。）

![](cnn17.png)

## CNN工业上的问题



# 附录

## 可视化
- TensorBoard:TensorFlow集成可视化工具
  GitHub官方项目：https://github.com/tensorflow/tensorflow/tree/master/tensorflow/tensorboard
  更多详细内容参考：
  - [TensorFlow中文社区] TensorBoard:可视化学习
  - [TensorFlow中文社区] TensorBoard:图表可视化
  - [极客学院] TensorBoard:可视化学习